In [1]:
!pip install sklearn_crfsuite jax[cpu]==0.4.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518703 sha256=d838485f70af40d92af6d13814f199647c385392db0a1c6d153db6024a97035e
  Stored in directory: /root/.cache/pip/wheels/f3/7a/25/f297f69029b5e4064e4736a0c4b3996a44cc27781c120bcb99
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33
ERROR: pip's dependency res

In [2]:
import os
import joblib
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('always') 

from glob import glob

from sklearn import preprocessing
from sklearn import model_selection
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split


import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [3]:
class config:
    DATASET = '/kaggle/input/vner-vlsp-2021/data/*'

# Load and explore VNER dataset

In [4]:
files = glob(config.DATASET)

In [5]:
names = ['Word', 'pos', 'phrase', 'NER', 'NER1', 'NER2']
df = pd.read_csv(files[0], sep='\t', quoting=3, names=names, skip_blank_lines=False)
df.head(10)

,Word,pos,phrase,NER,NER1,NER2
0,Mong ước,_,_,O,O,O
1,được,_,_,O,O,O
2,đến,_,_,O,O,O
3,trường,_,_,O,O,O
4,của,_,_,O,O,O
5,bé,_,_,O,O,O
6,gái,_,_,O,O,O
7,7,_,_,B-QUANTITY-AGE,O,O
8,tuổi,_,_,I-QUANTITY-AGE,O,O
9,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
concatenated = pd.DataFrame(columns=names)
for file in files:
    df = pd.read_csv(file, sep='\t', quoting=3, names=names,  skip_blank_lines=False)
    concatenated = pd.concat([concatenated, df], ignore_index=True)

In [7]:
dataset = concatenated[['Word', 'NER']]

# Preprocessing Data

In [8]:
dataset.loc[:, 'Word'] = dataset['Word'].replace(np.nan, 'NAN')
dataset.loc[:, 'NER'] = dataset['NER'].replace(np.nan, 'NAN')
# dataframe['NER_nested'] = dataframe['NER_nested'].replace(np.nan, 'NAN')

In [9]:
dataset = dataset[dataset['NER'].str.isupper()] #NER phải là chuỗi in HOA
# dataframe = dataframe[dataframe['NER_nested'].str.isupper()]

In [10]:
uppercase_rows = dataset['Word'].notna() & dataset['Word'].str.isupper() & dataset['Word'] == 'NAN'
dataset = dataset[~uppercase_rows]
dataset.head(10)

,Word,NER
0,Mong ước,O
1,được,O
2,đến,O
3,trường,O
4,của,O
5,bé,O
6,gái,O
7,7,B-QUANTITY-AGE
8,tuổi,I-QUANTITY-AGE
9,NAN,NAN


In [11]:
dataset.shape

(1008205, 2)

In [12]:
def create_col_sentence(dataset: pd.DataFrame):
    sentence_number = 1
    sentence_column = []
    for index, row in dataset.iterrows():
        if row['Word'] == 'NAN':
            sentence_number += 1
        sentence_column.append(f"Sentence: {sentence_number}")
    
    dataset['Sentence #'] = sentence_column
    dataset = dataset[dataset['Word'] != 'NAN']
    
    dataset = dataset.dropna(axis=0).reset_index(drop = True)
    return dataset

In [13]:
dataset = create_col_sentence(dataset)
dataset.head(10)

,Word,NER,Sentence #
0,Mong ước,O,Sentence: 1
1,được,O,Sentence: 1
2,đến,O,Sentence: 1
3,trường,O,Sentence: 1
4,của,O,Sentence: 1
5,bé,O,Sentence: 1
6,gái,O,Sentence: 1
7,7,B-QUANTITY-AGE,Sentence: 1
8,tuổi,I-QUANTITY-AGE,Sentence: 1
9,Giờ,B-DATETIME,Sentence: 2


In [14]:
dataset.rename(columns = {'NER':'Tag'}, inplace = True)

In [15]:
dataset['Tag'].value_counts()

Tag
O                         833546
B-PERSON                   15409
I-ORGANIZATION             11552
B-ORGANIZATION              9981
B-LOCATION-GPE              8831
                           ...  
I                             16
B-LOCATION-GPE HCM             6
A                              1
B-LOCATION-GPE-GEO             1
B-ORGANIZATION NN&PTNT         1
Name: count, Length: 90, dtype: int64

In [16]:
X = dataset.drop('Tag', axis=1)
X.head()

,Word,Sentence #
0,Mong ước,Sentence: 1
1,được,Sentence: 1
2,đến,Sentence: 1
3,trường,Sentence: 1
4,của,Sentence: 1


In [17]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = dataset.Tag.values

In [18]:
classes = np.unique(y)
classes = classes.tolist()
classes

['A',
 'B',
 'B-ADDRESS',
 'B-DATETIME',
 'B-DATETIME-DATE',
 'B-DATETIME-DATERANGE',
 'B-DATETIME-DURATION',
 'B-DATETIME-SET',
 'B-DATETIME-TIME',
 'B-DATETIME-TIMERANGE',
 'B-EMAIL',
 'B-EVENT',
 'B-EVENT-CUL',
 'B-EVENT-GAMESHOW',
 'B-EVENT-NATURAL',
 'B-EVENT-SPORT',
 'B-IP',
 'B-LOCATION',
 'B-LOCATION-GEO',
 'B-LOCATION-GPE',
 'B-LOCATION-GPE HCM',
 'B-LOCATION-GPE-GEO',
 'B-LOCATION-STRUC',
 'B-MISCELLANEOUS',
 'B-ORGANIZATION',
 'B-ORGANIZATION NN&PTNT',
 'B-ORGANIZATION-MED',
 'B-ORGANIZATION-SPORTS',
 'B-ORGANIZATION-STOCK',
 'B-PERSON',
 'B-PERSONTYPE',
 'B-PHONENUMBER',
 'B-PRODUCT',
 'B-PRODUCT-AWARD',
 'B-PRODUCT-COM',
 'B-PRODUCT-LEGAL',
 'B-QUANTITY',
 'B-QUANTITY-AGE',
 'B-QUANTITY-CUR',
 'B-QUANTITY-DIM',
 'B-QUANTITY-NUM',
 'B-QUANTITY-ORD',
 'B-QUANTITY-PER',
 'B-QUANTITY-TEM',
 'B-SKILL',
 'B-URL',
 'I',
 'I-ADDRESS',
 'I-DATETIME',
 'I-DATETIME-DATE',
 'I-DATETIME-DATERANGE',
 'I-DATETIME-DURATION',
 'I-DATETIME-SET',
 'I-DATETIME-TIME',
 'I-DATETIME-TIMERANGE',


In [19]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['A',
 'B',
 'B-ADDRESS',
 'B-DATETIME',
 'B-DATETIME-DATE',
 'B-DATETIME-DATERANGE',
 'B-DATETIME-DURATION',
 'B-DATETIME-SET',
 'B-DATETIME-TIME',
 'B-DATETIME-TIMERANGE',
 'B-EMAIL',
 'B-EVENT',
 'B-EVENT-CUL',
 'B-EVENT-GAMESHOW',
 'B-EVENT-NATURAL',
 'B-EVENT-SPORT',
 'B-IP',
 'B-LOCATION',
 'B-LOCATION-GEO',
 'B-LOCATION-GPE',
 'B-LOCATION-GPE HCM',
 'B-LOCATION-GPE-GEO',
 'B-LOCATION-STRUC',
 'B-MISCELLANEOUS',
 'B-ORGANIZATION',
 'B-ORGANIZATION NN&PTNT',
 'B-ORGANIZATION-MED',
 'B-ORGANIZATION-SPORTS',
 'B-ORGANIZATION-STOCK',
 'B-PERSON',
 'B-PERSONTYPE',
 'B-PHONENUMBER',
 'B-PRODUCT',
 'B-PRODUCT-AWARD',
 'B-PRODUCT-COM',
 'B-PRODUCT-LEGAL',
 'B-QUANTITY',
 'B-QUANTITY-AGE',
 'B-QUANTITY-CUR',
 'B-QUANTITY-DIM',
 'B-QUANTITY-NUM',
 'B-QUANTITY-ORD',
 'B-QUANTITY-PER',
 'B-QUANTITY-TEM',
 'B-SKILL',
 'B-URL',
 'I',
 'I-ADDRESS',
 'I-DATETIME',
 'I-DATETIME-DATE',
 'I-DATETIME-DATERANGE',
 'I-DATETIME-DURATION',
 'I-DATETIME-SET',
 'I-DATETIME-TIME',
 'I-DATETIME-TIMERANGE',


# Conditional Random Fields (CRFs)


In [20]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

## Get sentences


In [21]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        
        self.grouped = self.data.groupby('Sentence #').apply(agg_func, include_groups=False)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.sentences[self.n_sent - 1]
            self.n_sent += 1
            return s 
        except IndexError:
            return None

In [22]:
getter = SentenceGetter(dataset)
sentences = getter.sentences

## Features extraction

In [23]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

In [24]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [26]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

In [27]:
y_pred = crf.predict(X_test)

In [28]:
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels = new_classes, zero_division=0)

0.8999499957443855

In [29]:
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes, zero_division=0))

                        precision    recall  f1-score   support

                     A       0.00      0.00      0.00         1
                     B       1.00      1.00      1.00        19
             B-ADDRESS       1.00      1.00      1.00        29
            B-DATETIME       1.00      1.00      1.00      1133
       B-DATETIME-DATE       1.00      1.00      1.00       620
  B-DATETIME-DATERANGE       1.00      1.00      1.00       133
   B-DATETIME-DURATION       1.00      1.00      1.00       531
        B-DATETIME-SET       1.00      1.00      1.00        12
       B-DATETIME-TIME       1.00      1.00      1.00       113
  B-DATETIME-TIMERANGE       1.00      1.00      1.00       161
               B-EMAIL       1.00      1.00      1.00        12
               B-EVENT       1.00      1.00      1.00       192
           B-EVENT-CUL       1.00      1.00      1.00        66
      B-EVENT-GAMESHOW       1.00      1.00      1.00       109
       B-EVENT-NATURAL       1.00      

In [30]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted',
                        labels=classes, 
                        zero_division=0)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=10,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True,
                                 max_iterations=100),
                   n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78eef3cd3250>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78ef3104cd90>},
                   scoring=make_scorer(flat_f1_score, average=we...ION-MED', 'I-ORGANIZATION-SPORTS', 'I-ORGANIZATION-STOCK', 'I-PERSON', 'I-PERSONTYPE', 'I-PHONENUMBER', 'I-PRODUCT', 'I-PRODUCT-AWARD', 'I-PRODUCT-COM', 'I-PRODUCT-LEGAL', 'I-QUANTITY', 'I-QUANTITY-AGE', 'I-QUANTITY-CUR', 'I-QUANTITY-DIM', 'I-QUANTITY-NUM', 'I-QUANTITY-ORD', 'I-QUANTITY-PER', 'I-QUANTITY-TEM', 'I-SKILL', 'I-URL', 'NAN', 'O'], zero_division=0),
                   verbose=1)

In [31]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.7146912130313807, 'c2': 0.020176090276753964}
best CV score: 0.999997973238643
model size: 0.02M


In [32]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=classes, zero_division=0))

                        precision    recall  f1-score   support

                     A       0.00      0.00      0.00         1
                     B       1.00      1.00      1.00        19
             B-ADDRESS       1.00      1.00      1.00        29
            B-DATETIME       1.00      1.00      1.00      1133
       B-DATETIME-DATE       1.00      1.00      1.00       620
  B-DATETIME-DATERANGE       1.00      1.00      1.00       133
   B-DATETIME-DURATION       1.00      1.00      1.00       531
        B-DATETIME-SET       1.00      1.00      1.00        12
       B-DATETIME-TIME       1.00      1.00      1.00       113
  B-DATETIME-TIMERANGE       1.00      1.00      1.00       161
               B-EMAIL       1.00      1.00      1.00        12
               B-EVENT       1.00      1.00      1.00       192
           B-EVENT-CUL       1.00      1.00      1.00        66
      B-EVENT-GAMESHOW       1.00      1.00      1.00       109
       B-EVENT-NATURAL       1.00      

In [33]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s\t ->\t %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-PRODUCT-AWARD	 ->	 I-PRODUCT-AWARD 3.351098
B-QUANTITY-PER	 ->	 I-QUANTITY-PER 3.092353
B-MISCELLANEOUS	 ->	 I-MISCELLANEOUS 3.082203
B-DATETIME-TIME	 ->	 I-DATETIME-TIME 2.959941
I-ADDRESS	 ->	 I-ADDRESS 2.712866
B-LOCATION-STRUC	 ->	 I-LOCATION-STRUC 2.656710
I-DATETIME-DATERANGE	 ->	 I-DATETIME-DATERANGE 2.607271
B-PRODUCT-LEGAL	 ->	 I-PRODUCT-LEGAL 2.556445
I-DATETIME-SET	 ->	 I-DATETIME-SET 2.544235
I-PHONENUMBER	 ->	 I-PHONENUMBER 2.529645
B-DATETIME-TIMERANGE	 ->	 I-DATETIME-TIMERANGE 2.516760
I-IP  	 ->	 I-IP    2.478429
B-DATETIME	 ->	 I-DATETIME 2.406410
I-SKILL	 ->	 I-SKILL 2.389088
B-ORGANIZATION-MED	 ->	 I-ORGANIZATION-MED 2.387275
B-DATETIME-DATE	 ->	 I-DATETIME-DATE 2.354428
B-EVENT-GAMESHOW	 ->	 I-EVENT-GAMESHOW 2.292005
I-PRODUCT-AWARD	 ->	 I-PRODUCT-AWARD 2.276519
I-PRODUCT-LEGAL	 ->	 I-PRODUCT-LEGAL 2.272571

Top unlikely transitions:
I-PERSONTYPE	 ->	 I-PERSONTYPE 1.163543
I-MISCELLANEOUS	 ->	 I-MISCELLANEOUS 1.160597
I-QUANTITY-DIM	 ->	 I-

In [34]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
12.753832 B-PERSON postag:B-PERSON
12.603876 B-LOCATION-GPE postag:B-LOCATION-GPE
12.524992 B-ORGANIZATION postag:B-ORGANIZATION
12.495060 B-QUANTITY-NUM postag:B-QUANTITY-NUM
12.152864 B-LOCATION postag:B-LOCATION
12.049053 B-PERSONTYPE postag:B-PERSONTYPE
11.303629 I-ORGANIZATION postag:I-ORGANIZATION
11.211959 B-ORGANIZATION-SPORTS postag:B-ORGANIZATION-SPORTS
11.034909 B-QUANTITY postag:B-QUANTITY
10.963884 B-PRODUCT postag:B-PRODUCT
10.766550 I-PERSON postag:I-PERSON
10.753171 I-LOCATION-GPE postag:I-LOCATION-GPE
10.724329 I-LOCATION postag:I-LOCATION
10.694083 B-DATETIME postag:B-DATETIME
10.588783 B-QUANTITY-CUR postag:B-QUANTITY-CUR
10.559715 I-PERSONTYPE postag:I-PERSONTYPE
10.518267 B-QUANTITY-ORD postag:B-QUANTITY-ORD
10.480517 B-PRODUCT-COM postag:B-PRODUCT-COM
10.318689 B-DATETIME-DATE postag:B-DATETIME-DATE
10.307052 I-QUANTITY postag:I-QUANTITY
10.124157 B-DATETIME-DURATION postag:B-DATETIME-DURATION
10.067964 I-DATETIME-DURATION postag:I-DATETIME-DURATION


# ELI5
ELI5 is a Python package which helps to debug machine learning classifiers and explain their predictions. ELI5 allows to check weights of sklearn_crfsuite.CRF models.

In [35]:
import eli5
eli5.show_weights(crf, top=10)

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


From \ To,B,I,O,B-ADDRESS,I-ADDRESS,B-DATETIME,I-DATETIME,B-DATETIME-DATE,I-DATETIME-DATE,B-DATETIME-DATERANGE,I-DATETIME-DATERANGE,B-DATETIME-DURATION,I-DATETIME-DURATION,B-DATETIME-SET,I-DATETIME-SET,B-DATETIME-TIME,I-DATETIME-TIME,B-DATETIME-TIMERANGE,I-DATETIME-TIMERANGE,B-EMAIL,I-EMAIL,B-EVENT,I-EVENT,B-EVENT-CUL,I-EVENT-CUL,B-EVENT-GAMESHOW,I-EVENT-GAMESHOW,B-EVENT-NATURAL,I-EVENT-NATURAL,B-EVENT-SPORT,I-EVENT-SPORT,B-IP,I-IP,B-LOCATION,I-LOCATION,B-LOCATION-GEO,I-LOCATION-GEO,B-LOCATION-GPE,I-LOCATION-GPE,B-LOCATION-GPE HCM,B-LOCATION-STRUC,I-LOCATION-STRUC,B-MISCELLANEOUS,I-MISCELLANEOUS,B-ORGANIZATION,I-ORGANIZATION,B-ORGANIZATION NN&PTNT,B-ORGANIZATION-MED,I-ORGANIZATION-MED,B-ORGANIZATION-SPORTS,I-ORGANIZATION-SPORTS,B-ORGANIZATION-STOCK,I-ORGANIZATION-STOCK,B-PERSON,I-PERSON,B-PERSONTYPE,I-PERSONTYPE,B-PHONENUMBER,I-PHONENUMBER,B-PRODUCT,I-PRODUCT,B-PRODUCT-AWARD,I-PRODUCT-AWARD,B-PRODUCT-COM,I-PRODUCT-COM,B-PRODUCT-LEGAL,I-PRODUCT-LEGAL,B-QUANTITY,I-QUANTITY,B-QUANTITY-AGE,I-QUANTITY-AGE,B-QUANTITY-CUR,I-QUANTITY-CUR,B-QUANTITY-DIM,I-QUANTITY-DIM,B-QUANTITY-NUM,I-QUANTITY-NUM,B-QUANTITY-ORD,I-QUANTITY-ORD,B-QUANTITY-PER,I-QUANTITY-PER,B-QUANTITY-TEM,I-QUANTITY-TEM,B-SKILL,I-SKILL,B-URL,I-URL
B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O,0.0,0.0,0.119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-ADDRESS,0.0,0.0,0.0,0.0,2.172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-ADDRESS,0.0,0.0,0.0,0.0,2.713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATETIME,0.0,0.0,0.0,0.0,0.0,0.0,2.406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATETIME,0.0,0.0,0.0,0.0,0.0,0.0,0.937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATETIME-DATE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# Save & Load Model

In [36]:
import pickle

with open('/kaggle/working/crf_model.pkl', 'wb') as file:
    pickle.dump(crf, file)

In [37]:
with open('/kaggle/working/crf_model.pkl', 'rb') as file:
    crf_loaded = pickle.load(file)